In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
t1=time.time()
train = pd.read_csv('./daguan_data/train_set.csv')
test = pd.read_csv('./daguan_data/test_set.csv')
test_id = pd.read_csv('./daguan_data/test_set.csv')[["id"]].copy()

column="word_seg"
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
trn_term_doc = vec.fit_transform(train[column])
test_term_doc = vec.transform(test[column])

y=(train["classify"]-1).astype(int)
clf = LogisticRegression(C=4, dual=True)
clf.fit(trn_term_doc, y)
preds=clf.predict_proba(test_term_doc)

#保存概率文件
test_prob=pd.DataFrame(preds)
test_prob.columns=["class_prob_%s"%i for i in range(1,preds.shape[1]+1)]
test_prob["id"]=list(test_id["id"])
test_prob.to_csv('./daguan_data/prob_lr_baseline.csv',index=None)

#生成提交结果
preds=np.argmax(preds,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv('sub_lr_baseline.csv',index=None)
t2=time.time()
print("time use:",t2-t1)

KeyError: 'classify'

1、比赛任务描述
这次比赛的任务本质上就是一个长文本分类任务，目标是“建立模型通过长文本数据正文，预测文本对应的类别”，比赛的分类类别一共有19种。例如给定一篇新闻文章（News Articles），预测这篇文章的类别是Food，Sports还是Politics。


1.1 评分标准
采用各个类别F1指标的算术平均值。其中，Pi表示第i个类别对应的准确率，Ri表示第i个类别对应的召回率。


2、数据介绍
数据包含2个csv文件，train_set.csv和test_set.csv，分别用于训练模型和测试。值得注意的有以下几点：

数据每一行对应一篇文章，文章分别在“字”和“词”的级别上做了脱敏处理，也就是说数据中我们看不到具体的字和词，只有它们的数字id，每个id对应一个字或词。
一篇文章的文本数据包含两类，分别是文章正文在“字”级别上的表示article，和在“词”级别上的表示word。
2.1 数据分析
经过统计，train_set.csv和test_set.csv分别包含十万条数据，包含1万多个不同article和84万多个不同word。数据的平均字长度为1177，平均词长度为718，可以看出文本非常长，这也是本次比赛数据集最关键的特点。

2.2 数据预处理
将train_set.csv按照9:1的比例划分为训练集和验证集，应用深度学习模型需要将文本进行截断补齐，这里的截断补齐长度分别为2000和3200，采用的策略是要求这个长度能够覆盖95%的数据。


文本预处理部分用到了torchtext，torchtext是专门用于文本预处理的python包，使用简单，可以节省很多文本预处理部分的工作，具体的使用参考比赛源码和官方文档pytorch/text。

2.3 预训练字/词向量
因为这次的文本数据经过脱敏，因此无法利用外部的预训练字/词向量，需要我们自己训练。我们使用word2vec包：danielfrg/word2vec进行预训练，向量维度为300维。（赛后听其他组的分享，分别用word2vec，glove，fasttext训练100维向量，然后拼接成300维作为字/词向量，能获得更好的模型训练结果）

3、模型介绍
基于深度学习的文本分类中总结：

深度学习文本分类模型概括来说，主要有五大类模型：
1、词嵌入向量化：word2vec，FastText等
2、卷积神经网络特征提取：Text-CNN，Char-CNN等
3、上下文机制：Text-RNN，BiRNN，RCNN等
4、记忆存储机制：EntNet，DMN等
5、注意力机制：HAN等
其中我们采用的模型有五种：

TextCNN
GRU
RCNN
FastText
Attention-GRU
模型的具体介绍建议仔细阅读知乎专栏文章用深度学习（CNN RNN Attention）解决大规模文本分类问题 - 综述和实践和基于深度学习的文本分类，这里我就不一一介绍了，值得注意的是这些模型的总体架构都是类似的：都是先将文本送入embedding层，然后通过CNN/GRU/RCNN等模型提取特征，最终通过分类器输出19维向量，分别对应19个类别的分类概率。


3.1 模型实现
深度学习模型实现我们采用了PyTorch，实现代码见比赛源码moneyDboat/data_grand中models文件夹，作为Pytorch的忠实拥趸，我之后会专门写一篇文章详细介绍说明，Pytorch相较于其他深度学习框架例如tensorflow、caffe等的优势，简单来说有这么几点：

简洁易用：Pytorch的设计追求最少的封装，尽量避免重复造轮子。
动态图实现：代码简洁直观，同时让调试更加简单。
易于学习掌握：Pytorch提供了清晰的文档和指南，以及论坛PyTorch Forums，遇到的问题基本上都能在论坛中查到。

3.2 传统机器学习模型
因为这次的文本非常长，所以传统的机器学习模型也能取得非常不错的效果，tfidf+svm的baseline在测试集上的f1就能达到0.779。

3.3 训练策略
训练策略很关键，参考知乎“看山杯” 夺冠记：

1、刚开始训练的时候Embedding层的学习率为0，其它层的学习率为1e-3，采用Adam优化器（一开始的时候卷积层都是随机初始化的，反向传播得到的Embedding层的梯度受到卷积层的影响，相当于噪声）
2、训练1-2个epoch之后，Embedding层的学习率设为2e-4
3、每个epoch或者半个epoch统计一次在验证集的分数：
如果分数上升，保存模型，并记下保存路径；如果分数下降，加载上一个模型的保存路径，并降低学习率为一半（重新初始化优化器，清空动量信息，而不是只修改学习率----使用PyTorch的话新建一个新优化器即可）
预训练字/词向量在模型训练过程中的调整非常必要，因为word2vec等其他预训练方法都是通过无监督训练得到的，没有利用到训练集的分类类别信息。实验表明，embedding层设置学习率之后，模型的f1值大约能提升3个百分点。

3.4 单模型结果
时间所限，另外一个模型训练完需要超过十个小时，我们没有进行特别细致的调参，最后得到的单模型结果如下：


4、模型融合
模型融合部分我们尝试过多种方法，最终还是采用过了最简单但效果非常稳定的概率等权重融合。将上述深度学习模型以及传统机器学习模型融合后线上得分能达到0.7970。


做到这里，我们陷入了瓶颈，再怎么用原来的模型做融合也没办法提高线上的分数，寻找一些其他有效的突破口。关于模型融合，周志华教授的西瓜书中提到：

模型融合中个体学习器需要“好而不同”。
就是说模型之间差异性越大，融合效果越好。那么如何创造差异性，有两种方法：

尝试不同模型
改变模型输入
其中第一种方法“尝试不同模型”，因为当时我们距离比赛结束只剩下不到三天的时间，再去实现新的模型不太现实，所以我们考虑采用第二种方法：改变模型输入。经过尝试，我们发现两种有效的方法：

重新划分训练集、验证集，用新的训练集训练数据
将文本倒序之后训练新的模型
两种方法都是考虑到原本的验证集和文本预处理中被截断的部分，损失了训练集的部分信息，通过这两种方式训练新的模型用于模型融合，最终的线上分数能提升近3个千分点，达到0.79995，让我们得以进入Top10。


5、总结
先谈一下自己的体会，这次是我第一次参加算法类的比赛，前后总共花了3个星期左右的时间完成比赛，这三个星期中确实非常辛苦，当然最辛苦的大概是服务器哈哈，从开始到比赛结束都没有休息过，花的电费应该也不少。自己基本上都是白天写代码，晚上回宿舍前在服务器上跑模型，第二天早上第一件事就是查看模型训练结果，看看一晚上的收成怎么样，然后继续改代码调参数，就这样不断循环往复，好在最后的比赛结果还算满意，自己也收获了很多经验。